In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,148054.02,-0.97,-0.10,0.50,-0.01,-0.06,0.01,-0.00,-0.00
3,-0.97,31694.61,-0.17,0.08,-0.07,0.01,0.02,0.00,0.00
4,-0.10,-0.17,6645.04,0.10,-0.00,-0.00,-0.01,-0.00,0.00
5,0.50,0.08,0.10,1343.38,0.00,-0.01,0.01,0.00,0.00
6,-0.01,-0.07,-0.00,0.00,256.31,-0.00,-0.00,0.00,0.00
7,-0.06,0.01,-0.00,-0.01,-0.00,46.56,0.00,0.00,0.00
8,0.01,0.02,-0.01,0.01,-0.00,0.00,7.99,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.26,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,-0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00001
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00003,-0.00001,0.00001
5,0.00004,0.00001,0.00003,1.00000,0.00001,-0.00002,0.00005,0.00000,0.00001
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00001,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.5786916853882604

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.43746389e+09 1.09898342e+08 4.73225143e+06 1.88051780e+05
 6.57355892e+03 2.03467786e+02 5.33637522e+00 9.92471018e-02
 9.07821506e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999591,-0.028011,-0.005582,-0.001123,-0.000215,-0.000039,-0.000007,-0.000001,-1.287922e-07
1,0.027815,0.999099,-0.031455,-0.006048,-0.001154,-0.000209,-0.000035,-0.000006,-6.624660e-07
2,0.006407,0.031048,0.998854,-0.035258,-0.006421,-0.001164,-0.000201,-0.000031,-3.718646e-06
3,0.001504,0.007050,0.034754,0.998554,-0.039757,-0.006921,-0.001181,-0.000186,-2.219286e-05
4,0.000345,0.001612,0.007688,0.039122,0.998061,-0.047123,-0.007764,-0.001217,-1.454637e-04
5,0.000078,0.000365,0.001740,0.008618,0.046256,0.997103,-0.059060,-0.008923,-1.065126e-03
6,0.000018,0.000082,0.000398,0.001957,0.010286,0.057710,0.994992,-0.080399,-9.259377e-03
7,0.000004,0.000019,0.000093,0.000458,0.002402,0.013236,0.078058,0.989636,-1.197742e-01
8,0.000001,0.000005,0.000022,0.000111,0.000581,0.003198,0.018633,0.118638,9.927574e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00040862429639365594,
 0.0009009124789138845,
 0.001146019073639648,
 0.001445935293966194,
 0.0019392464368649387,
 0.002897104518622129,
 0.00500757177083011,
 0.010363782612550998,
 0.007242601999981835]